# NLP - Homework 2 - task a with BiLSTM



## Setup

In [ ]:
!pip install torchtext --upgrade
# install lightning
!pip install pytorch_lightning &> /dev/null

Requirement already up-to-date: torchtext in /usr/local/lib/python3.7/dist-packages (0.9.1)


## Importing libraries

In [ ]:
# here go all the imports
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import Vectors
import pytorch_lightning as pl


import nltk

from pprint import pprint
from tqdm import tqdm
from torchtext.vocab import Vocab
from collections import Counter
from typing import *
from string import punctuation
import re

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Allow reproducibility

In [ ]:
SEED = 1234

# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
pl.seed_everything(SEED, workers=True)

Global seed set to 1234


1234

## Importing data

In [ ]:
# Let's create a new folder and download the dataset.
! mkdir data
! wget -O data/laptops_train.json https://raw.githubusercontent.com/SapienzaNLP/nlp2021-hw2/master/data/laptops_train.json
! wget -O data/laptops_dev.json https://raw.githubusercontent.com/SapienzaNLP/nlp2021-hw2/master/data/laptops_dev.json
! wget -O data/restaurants_train.json https://raw.githubusercontent.com/SapienzaNLP/nlp2021-hw2/master/data/restaurants_train.json
! wget -O data/restaurants_dev.json https://raw.githubusercontent.com/SapienzaNLP/nlp2021-hw2/master/data/restaurants_dev.json

mkdir: cannot create directory ‘data’: File exists
--2021-06-10 15:26:14--  https://raw.githubusercontent.com/SapienzaNLP/nlp2021-hw2/master/data/laptops_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 668974 (653K) [text/plain]
Saving to: ‘data/laptops_train.json’

data/laptops_train. 100%[===================>] 653.29K  --.-KB/s    in 0.05s   

2021-06-10 15:26:14 (13.1 MB/s) - ‘data/laptops_train.json’ saved [668974/668974]

--2021-06-10 15:26:14--  https://raw.githubusercontent.com/SapienzaNLP/nlp2021-hw2/master/data/laptops_dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... 

In [ ]:
import pandas as pd
import os
import json

root_folder: str = r"./data" # to save dataset

laptops_train_filename = "laptops_train.json"
laptops_valid_filename = "laptops_dev.json"
restaurants_train_filename = "restaurants_train.json"
restaurants_valid_filename = "restaurants_dev.json"

laptops_train_dataframe = pd.read_json(os.sep.join([root_folder, laptops_train_filename]), orient='records')
laptops_valid_dataframe = pd.read_json(os.sep.join([root_folder, laptops_valid_filename]), orient='records')
restaurants_train_dataframe = pd.read_json(os.sep.join([root_folder, restaurants_train_filename]), orient='records')
restaurants_valid_dataframe = pd.read_json(os.sep.join([root_folder, restaurants_valid_filename]), orient='records')

restaurants_train_dataframe.drop('categories', inplace=True, axis=1)
restaurants_valid_dataframe.drop('categories', inplace=True, axis=1)

train_dataframe = pd.concat([laptops_train_dataframe, restaurants_train_dataframe], ignore_index=True)
valid_dataframe = pd.concat([laptops_valid_dataframe, restaurants_valid_dataframe], ignore_index=True)

laptops_train_dataframe = None
laptops_valid_dataframe = None
restaurants_train_dataframe = None
restaurants_valid_dataframe = None

VALID_SIZE = len(valid_dataframe)
TEST_SIZE = VALID_SIZE

train_dataframe

,targets,text
0,"[[[22, 31], hard disk, neutral]]",I always use a backup hard disk to store impor...
1,"[[[34, 38], size, positive]]","I also love the small, convenient size of my l..."
2,"[[[76, 84], mousepad, neutral]]",I thought the white Mac computers looked dirty...
3,"[[[40, 48], responds, positive]]","It is always reliable, never bugged and respon..."
4,"[[[55, 63], keyboard, positive], [[73, 78], sp...",The real stand out on this computer is the fee...
...,...,...
4995,[],I actually gave Patroon another chance before ...
4996,"[[[77, 84], service, positive], [[123, 128], p...",Although they do the typical what kind of wate...
4997,[],"That said, I thought Scalini Fedeli was one of..."
4998,"[[[27, 34], waiters, positive], [[125, 131], d...",Have always found that the waiters will go out...


In [ ]:
valid_dataframe

,targets,text
0,[],It was over rated!
1,"[[[55, 75], adding the bluetooth, negative]]",But Sony said we could send it back and be cha...
2,"[[[4, 21], Windows 7 Starter, positive]]","The Windows 7 Starter is, in my opinion, a gre..."
3,"[[[4, 14], powerpoint, positive]]",The powerpoint opened seamlessly in the apple ...
4,"[[[74, 80], screen, positive]]","I chose the iBookG4, a laptop that is an attra..."
...,...,...
1081,[],"(or sister, in my case!)."
1082,"[[[0, 5], Staff, positive]]",Staff is very accomodating.
1083,"[[[26, 40], yellowfun tuna, positive], [[51, 6...",I particularly love their yellowfun tuna and t...
1084,[],Enjoy!


In [ ]:
! wget http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip -q glove.840B.300d.zip -d ./data

--2021-06-10 15:26:16--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-06-10 15:26:16--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-06-10 15:26:16--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [ ]:
embs_file_name: str = r"glove.840B.300d.txt"
vectors = Vectors(os.sep.join([root_folder, embs_file_name]))

In [ ]:
def spans_for_test(txt: str) -> List[Tuple[int, int]]:
    # This function returns the spans of the several tokens of the input text
    tokens = nltk.word_tokenize(txt)
    tokens = filter(lambda token: preprocess_term_for_test(token) != '', tokens)
    offset = 0
    spans_list = list()
    
    for token in tokens:
        
        if token == '``' or token == "''":
          token = '"'
        elif token[-1] in punctuation and len(token) > 2:
          token = token[:-1]
        if token[0] in punctuation and len(token) > 2:
          token = token[1:]

        filtered_punctuation = punctuation.replace('"', '').replace('.', '').replace("'", '')
        tokens_list = re.split("[" + filtered_punctuation + "]+", token)
        for token in tokens_list:
          if token == '':
            continue
          offset = txt.find(token, offset)
          spans_list.append((offset, offset+len(token)))
          offset += len(token)
    
    return spans_list



def preprocess_term_for_test(term: str) -> str: # I remove punctuation
    # This function is used in order to preprocess a signle tokenized term
    cleaned_term = ''
    filtered_punctuation = punctuation.replace('`', '')
    for char in term:
      if (char not in filtered_punctuation) and (char not in '“”'):
        cleaned_term = cleaned_term + char

    return cleaned_term

def preprocess_text_for_test(text: str, start_indexes: List[int], end_indexes: List[int]) -> List[Dict[str, Union[str, Tuple[int, int]]]]:
  # This function returns a list of dicts, where each dict is associated to a tokenized preprocessed term and contains information about the token itslef, its label and its span indexes
  start_indexes.sort()
  end_indexes.sort()
  AT_indexes = list(zip(start_indexes, end_indexes))

  return list(map(lambda indexes: {'token': text[indexes[0] : indexes[1]].lower(), 'ne_label': 'AT' if any((indexes[0] >= true_indexes[0] and indexes[1] <= true_indexes[1]) for true_indexes in AT_indexes) else 'O', 'indexes': (indexes[0] , indexes[1])}, spans_for_test(text)))

In [ ]:
# These functions are similar to those above, but here a split of the text is made considering that you already know the information of the position of the aspect terms within the input text

def spans_for_train(txt: str) -> List[Tuple[int, int]]:
    tokens = nltk.word_tokenize(txt)
    tokens = filter(lambda token: preprocess_term_for_train(token) != '', tokens)
    offset = 0
    spans_list = list()
    for token in tokens:
        offset = txt.find(token, offset)
        spans_list.append((offset, offset+len(token)))
        offset += len(token)
    return spans_list

def preprocess_term_for_train(term: str) -> str: # I remove punctuation
    cleaned_term = ''
    for char in term:
      if (char not in punctuation) and (char not in '“”'):
        cleaned_term = cleaned_term + char

    return cleaned_term

def preprocess_text_for_train(text: str, start_indexes: List[int], end_indexes: List[int]) -> List[Dict[str, Union[str, Tuple[int, int]]]]:
  preprocessed_text = []
  end_indexes.insert(0, 0)
  start_indexes.sort()
  end_indexes.sort()
  
  for i in range(len(start_indexes)):
    start_index = start_indexes[i]
    end_index = end_indexes[i + 1]
    preprocessed_text += list(map(lambda indexes: {'token': text[end_indexes[i] + indexes[0] : end_indexes[i] + indexes[1]].lower(), 'ne_label': 'O', 'indexes': (end_indexes[i] + indexes[0] , end_indexes[i] + indexes[1])}, spans_for_train(text[end_indexes[i] : start_index])))
    aspect_term = text[start_index:end_index].split(' ')
    aspect_term_list = []
    for i in range(len(aspect_term)):
      term = aspect_term[i]
      preprocessed_term = preprocess_term_for_train(term)
      if preprocessed_term != '':
        d = {'token': preprocessed_term, 'ne_label': 'AT', 'indexes': (start_index, start_index + len(term))}
        aspect_term_list.append(d)
      start_index += len(term) + 1
    preprocessed_text += aspect_term_list
  preprocessed_text += list(map(lambda indexes: {'token': text[end_indexes[-1] + indexes[0] : end_indexes[-1] + indexes[1]].lower(), 'ne_label': 'O', 'indexes': (end_indexes[-1] + indexes[0] , end_indexes[-1] + indexes[1])}, spans_for_train(text[end_indexes[-1] : ])))

  return preprocessed_text

In [ ]:
class ABSA_A_Dataset(Dataset):

    def __init__(self, 
                 targets: pd.Series,
                 text: pd.Series,
                 preprocess_text: Callable[[str, List[int], List[int]], List[Dict[str, Union[str, Tuple[int, int]]]]]=preprocess_text_for_train):
        """
        We assume that the dataset can fit in memory.
        Args:
            targets (pd.Series): Pandas Dataframe column containing the targets.
            text (pd.Series): Pandas Dataframe column containing the text.
            preprocess_text (Callable): function that is used in order to preprocess the input texts
        """

        self.data = []
        for row_targets, row_text in zip(targets, text):
          start_indexes, end_indexes, true_aspect_terms = [], [], []
          for target in row_targets:
            indexes = target[0]
            start_indexes.append(indexes[0])
            end_indexes.append(indexes[1])
            true_aspect_terms.append(target[1])
          preprocessed_text = preprocess_text(row_text, start_indexes, end_indexes)
          self.data.append({"text": row_text,
                            "preprocessed_text": preprocessed_text,
                            "true_aspect_terms": true_aspect_terms})
        self.encoded_data = None
    
    def index_dataset(self, l_vocabulary, l_label_vocabulary):
        self.encoded_data = list()
        for i in range(len(self.data)):
            # for each sentence
            data_i = self.data[i]
            elem = data_i["preprocessed_text"]
            encoded_elem = torch.LongTensor(self.encode_text(elem, l_vocabulary))
            # for each element d in elem (d is a dictionary) 
            encoded_labels = torch.LongTensor([l_label_vocabulary[d["ne_label"]] for d in elem])
            indexes_list = [w["indexes"] for w in elem]

            self.encoded_data.append({"inputs": encoded_elem, 
                                      "outputs": encoded_labels,
                                      "text": data_i["text"],
                                      "indexes": indexes_list,
                                      "true_aspect_terms": data_i["true_aspect_terms"]})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.encoded_data is None:
            raise RuntimeError("""Trying to retrieve elements but index_dataset
            has not been invoked yet! Be sure to invoce index_dataset on this object
            before trying to retrieve elements. In case you want to retrieve raw
            elements, use the method get_raw_element(idx)""")
        return self.encoded_data[idx]
    
    def get_raw_element(self, idx):
        return self.data[idx]

    @staticmethod
    def encode_text(sentence:list, 
                l_vocabulary:Vocab):
        """
        Args:
            sentences (list): list of Dicts, each carrying the information about
            one token.
            l_vocabulary (Vocab): vocabulary with mappings from words to indexes and viceversa.
        Return:
            The method returns a list of indexes corresponding to the input tokens.
        """
        indexes = list()
        for w in sentence:
            if w["token"] in l_vocabulary.stoi: # vocabulary string to integer
                indexes.append(l_vocabulary[w["token"]])
            else:
                indexes.append(l_vocabulary.unk_index)
        return indexes
    
    @staticmethod
    def decode_output(max_indices:List[List[int]],
                    l_label_vocabulary: Vocab):
        """
        Args:
            max_indices: a List where the i-th entry is a List containing the
            indexes preds for the i-th sample
            l_label_vocabulary (Vocab): It is the vocabulary containing the mapping from
            a string label to its corresponding index and vice versa
        Output:
            The method returns a list of batch_size length where each element is a list
            of labels, one for each input token.
        """
        predictions = list()
        for indices in max_indices:
            # vocabulary integer to string is used to obtain the corresponding word from the max index
            predictions.append([l_label_vocabulary.itos[i] for i in indices])
        return predictions

In [ ]:
def build_vocab(dataset, min_freq=1):
    counter = Counter()
    for i in tqdm(range(len(dataset))):
        preprocessed_text = dataset.get_raw_element(i)["preprocessed_text"]
        # for each token in the sentence viewed as a dictionary
        for token in preprocessed_text:
          if token["token"] in vectors.stoi:
            counter[token["token"]]+=1
    for token in vectors.stoi:
      counter[token] += 1
    # we add special tokens for handling padding and unknown words at testing time.
    return Vocab(counter, specials=['<pad>', '<unk>'], min_freq=min_freq)

def build_label_vocab(dataset):
    counter = Counter()
    for i in tqdm(range(len(dataset))):
        preprocessed_text = dataset.get_raw_element(i)["preprocessed_text"]
        for token in preprocessed_text:
              counter[token["ne_label"]]+=1
    # No <unk> token for labels.
    return Vocab(counter, specials=['<pad>'])

dataset = ABSA_A_Dataset(train_dataframe['targets'], train_dataframe['text'])
vocabulary = build_vocab(dataset)
label_vocabulary = build_label_vocab(dataset)
dataset.index_dataset(vocabulary, label_vocabulary)

100%|██████████| 5000/5000 [00:00<00:00, 163847.68it/s]


In [ ]:
len(vectors), len(vocabulary)

(2196017, 2196018)

In [ ]:
import torchtext
print(torchtext.__version__)
pprint([f"{x}:{y}" for x, y in list(vocabulary.stoi.items())[:10]])
print("home index: ", vocabulary["home"])
print("<pad> index: ", vocabulary["<pad>"])
print("<unk> index", vocabulary["<unk>"])
print("word at index 52: ", vocabulary.itos[52])
print("unknown words are indexed at: ", vocabulary["alskfj"])

0.9.1
['<pad>:0',
 '<unk>:1',
 'the:2',
 'and:3',
 'i:4',
 'to:5',
 'a:6',
 'it:7',
 'is:8',
 'of:9']
home index:  322
<pad> index:  0
<unk> index 1
word at index 52:  service
unknown words are indexed at:  1


In [ ]:
print("vocab size:", len(label_vocabulary))
print("string to index")
pprint(label_vocabulary.stoi)
print()
print("index to string")
pprint(label_vocabulary.itos)

vocab size: 3
string to index
defaultdict(None, {'<pad>': 0, 'O': 1, 'AT': 2})

index to string
['<pad>', 'O', 'AT']


In [ ]:
# inputs from the first sentence
input_tensor = dataset[0]["inputs"]

# forms of the first sentence
raw_element = dataset.get_raw_element(0)["preprocessed_text"]
print([d["token"] for d in raw_element])
# Let’s print each token with the index in the vocabulary, ‘None’ stands for padding.
pprint(list(zip([d["token"] for d in raw_element], input_tensor.tolist())))

['i', 'always', 'use', 'a', 'backup', 'hard', 'disk', 'to', 'store', 'important', 'files', 'at', 'all', 'times']
[('i', 4),
 ('always', 102),
 ('use', 68),
 ('a', 6),
 ('backup', 1807),
 ('hard', 172),
 ('disk', 723),
 ('to', 5),
 ('store', 470),
 ('important', 802),
 ('files', 685),
 ('at', 31),
 ('all', 39),
 ('times', 120)]


In [ ]:
dataset = None # To free up RAM

In [ ]:
def rnn_collate_fn(
    data_elements: List[Dict[str, Union[torch.Tensor, List]]]) -> List[Dict[str, Union[torch.Tensor, List]]]:

    X, Y = [de['inputs'] for de in data_elements], [de['outputs'] for de in data_elements] # lists of index tensors
    
    batch = {}
    batch['inputs'] = pad_sequence(X, batch_first=True, padding_value=vocabulary['<pad>'])
    batch['outputs'] = pad_sequence(Y, batch_first=True, padding_value=label_vocabulary['<pad>'])
    batch['text'] = [de['text'] for de in data_elements]
    batch['indexes'] = [de['indexes'] for de in data_elements]
    batch['true_aspect_terms'] = [de['true_aspect_terms'] for de in data_elements]
    
    return batch

In [ ]:
class DataModuleABSA_A(pl.LightningDataModule):
    def __init__(self, training_targets, training_text, dev_targets, dev_text, vocabulary, label_vocabulary, batch_size=128, number_of_valid_samples=VALID_SIZE,
                 number_of_test_samples=TEST_SIZE):
        super().__init__()
        self.training_targets = training_targets
        self.training_text = training_text
        self.dev_targets = dev_targets
        self.dev_text = dev_text
        self.vocabulary = vocabulary
        self.label_vocabulary = label_vocabulary
        self.batch_size = batch_size
        self.number_of_valid_samples = number_of_valid_samples
        self.number_of_test_samples = number_of_test_samples

    def setup(self, stage=None):
      self.trainingset = ABSA_A_Dataset(self.training_targets, self.training_text)
      self.devset = ABSA_A_Dataset(self.dev_targets, self.dev_text, preprocess_text=preprocess_text_for_test) # Since here we are at test time, we cannot consider the information of the position of the aspect terms in the input texts,
      self.testset = ABSA_A_Dataset(self.dev_targets, self.dev_text, preprocess_text=preprocess_text_for_test) #so we have to change the preprocessing function.

      self.trainingset.index_dataset(self.vocabulary, self.label_vocabulary)
      self.devset.index_dataset(self.vocabulary, self.label_vocabulary)
      self.testset.index_dataset(self.vocabulary, self.label_vocabulary)
          
    def train_dataloader(self):
      return DataLoader(self.trainingset,
                        batch_size=self.batch_size,
                        shuffle=True, # We shuffle data at training time, at test time it is not needed
                        collate_fn=rnn_collate_fn,
                        num_workers=0)
    
    def val_dataloader(self):
        return DataLoader(self.devset,
                          batch_size=self.number_of_valid_samples,
                          shuffle=False,
                          collate_fn=rnn_collate_fn,
                          num_workers=0)
    
    def test_dataloader(self):
        return DataLoader(self.testset,
                          batch_size=self.number_of_test_samples,
                          shuffle=False,
                          collate_fn=rnn_collate_fn,
                          num_workers=0)

## Model Building

In [ ]:
x_str = "this is a nice example to explain word embeddings in pytorch ."
x_ids = [vocabulary.stoi[w] for w in x_str.split()]
print("sentence: ", x_str)
print("indexed sentence: ", x_ids)

sentence:  this is a nice example to explain word embeddings in pytorch .
indexed sentence:  [12, 8, 6, 115, 1909, 5, 4459, 768, 1716405, 13, 1, 25938]


In [ ]:
class AspectTermIdentificationModel(nn.Module):
    # we provide the hyperparameters as input
    def __init__(self, hparams, embeddings = None):
        super(AspectTermIdentificationModel, self).__init__()
        # Embedding layer: a mat∂rix vocab_size x embedding_dim where each index 
        # correspond to a word in the vocabulary and the i-th row corresponds to 
        # a latent representation of the i-th word in the vocabulary.
        pprint(hparams)
        self.input_padding_idx = hparams.input_padding_idx
        self.word_embedding = nn.Embedding(hparams.vocab_size, hparams.embedding_dim, padding_idx=self.input_padding_idx)
        if embeddings is not None:
            print("initializing embeddings from pretrained")
            self.word_embedding.weight.data.copy_(embeddings)
            self.word_embedding.weight.requires_grad = False # We doesn't train the embeddings

        # LSTM layer: an LSTM neural network that process the input text
        # (encoded with word embeddings) from left to right and outputs 
        # a new **contextual** representation of each word that depend
        # on the preciding words.
        self.lstm = nn.LSTM(hparams.embedding_dim, hparams.hidden_dim, 
                            bidirectional=hparams.bidirectional,
                            num_layers=hparams.num_layers, 
                            dropout = hparams.dropout if hparams.num_layers > 1 else 0,
                            batch_first=True)
        # Hidden layer: transforms the input value/scalar into
        # a hidden vector representation.
        lstm_output_dim = hparams.hidden_dim if hparams.bidirectional is False else hparams.hidden_dim * 2

        # During training, randomly zeroes some of the elements of the 
        # input tensor with probability hparams.dropout using samples 
        # from a Bernoulli distribution. Each channel will be zeroed out 
        # independently on every forward call.
        # This has proven to be an effective technique for regularization and 
        # preventing the co-adaptation of neurons
        self.dropout = nn.Dropout(hparams.dropout)
        self.classifier = nn.Linear(lstm_output_dim, hparams.num_classes)

    
    def forward(self, x):
        # Compute lengths of sequences, which we will need
        lengths = (x != self.input_padding_idx).long().sum(-1)
        lengths = lengths.to('cpu') # For the pack_padded_sequence method, it is required to be on CPU
        embeddings = self.word_embedding(x)
        embeddings = self.dropout(embeddings)
        embeddings = pack_padded_sequence(
            embeddings,
            lengths=lengths,
            batch_first=True,
            enforce_sorted=False)
        o, (h, c) = self.lstm(embeddings)
        o, _ = pad_packed_sequence(o, batch_first=True, padding_value=0.)
        o = self.dropout(o)
        output = self.classifier(o)
        return output

In [ ]:
def evaluate_extraction(samples: List[List[str]],
                        predictions_b: List[List[str]]) -> Tuple[Dict[str, int], float, float, float]:
    scores = {"tp": 0, "fp": 0, "fn": 0}
    tps, fps, fns = [], [], []
    for label, pred in zip (samples, predictions_b):
        pred_terms = {term_pred for term_pred in pred}
        gt_terms = {term_gt for term_gt in label}

        scores["tp"] += len(pred_terms & gt_terms)
        scores["fp"] += len(pred_terms - gt_terms)
        scores["fn"] += len(gt_terms - pred_terms)
        
    precision = 100 * scores["tp"] / (scores["tp"] + scores["fp"])
    recall = 100 * scores["tp"] / (scores["tp"] + scores["fn"])
    f1 = 2 * precision * recall / (precision + recall)
    
    return scores, precision, recall, f1

'''

def get_aspect_terms(labels, text, indexes):
  start_index = None
  aspect_terms = []
  punctuation_without_hyphen = punctuation.replace('-', '').replace('"', '').replace('/', '').replace('.', '').replace("'", '').replace('+', '')

  for i in range(len(labels)):
    if labels[i] == 'B-AT':
      if start_index is not None:
        aspect_terms_list = re.split("[" + punctuation_without_hyphen + "]+", text[start_index : end_index])
        for aspect_term in aspect_terms_list:
          if aspect_term != '':
            aspect_terms.append(aspect_term.strip())
      start_index, end_index = indexes[i]
    elif labels[i] == 'I-AT':
      end_index = indexes[i][1]
      if start_index is None:
        start_index = indexes[i][0]
  if start_index != None:
    aspect_terms_list = re.split("[" + punctuation_without_hyphen + "]+", text[start_index : end_index])
    for aspect_term in aspect_terms_list:
      if aspect_term != '':
        aspect_terms.append(aspect_term.strip())
  return aspect_terms

'''

def get_aspect_terms(labels: List[str],
                     text: str,
                     indexes: List[Tuple[int, int]]) -> List[str]:
  aspect_terms = []
  start_index = None
  last_seen_aspect_term_appended = False
  filtered_punctuation = punctuation.replace('-', '').replace('"', '').replace('/', '').replace('.', '').replace("'", '').replace('+', '')

  for i in range(len(labels)):

    if labels[i] == 'AT':
      if start_index is None:
        start_index = indexes[i][0]
      end_index = indexes[i][1]
      last_seen_aspect_term_appended = False

    elif not last_seen_aspect_term_appended and start_index is not None:
      aspect_terms_list = re.split("[" + filtered_punctuation + "]+", text[start_index : end_index])
      for aspect_term in aspect_terms_list:
        if aspect_term != '':
          aspect_terms.append(aspect_term.strip())
      start_index = None
      last_seen_aspect_term_appended = True

  if not last_seen_aspect_term_appended and start_index is not None:
      aspect_terms_list = re.split("[" + filtered_punctuation + "]+", text[start_index : end_index])
      for aspect_term in aspect_terms_list:
        if aspect_term != '':
          aspect_terms.append(aspect_term.strip())
  
  return aspect_terms

def evaluate_aspect_terms(decoded_labels: List[List[str]],
                          texts: List[str],
                          indexes: List[List[Tuple[int, int]]],
                          true_aspect_terms: List[List[str]]) -> Tuple[Dict[str, int], float, float, float]:
  gt = []
  preds = []
  for i in range(len(decoded_labels)):
    text = texts[i]
    indexes_list = indexes[i]
    predicted_aspect_terms = get_aspect_terms(decoded_labels[i], text, indexes_list)
    preds.append(predicted_aspect_terms)
    gt.append(true_aspect_terms[i])
  return evaluate_extraction(gt, preds)

In [ ]:
# install the pytorch CRF package
!pip install pytorch-crf

In [ ]:
import torch.optim as optim
from torchmetrics import F1
from torchcrf import CRF


class AspectTermModule(pl.LightningModule):
    def __init__(self, hparams, embeddings = None, *args, **kwargs):
        super(AspectTermModule, self).__init__(*args, **kwargs)

        self.save_hyperparameters(hparams)
        self.F1 = F1(num_classes=self.hparams.num_classes, average='macro', ignore_index=self.hparams.label_padding_idx)
        self.model = AspectTermIdentificationModel(self.hparams, embeddings)
        # crf layer
        self.crf = CRF(self.hparams.num_classes, batch_first=True)
    
    # This performs a forward pass of the model, as well as returning the predicted index.
    def forward(self, x, mask):
        logits = self.model(x)
        predictions = self.crf.decode(logits, mask)
        return logits, predictions

    # This runs the model in training mode mode, ie. activates dropout and gradient computation. It defines a single training step.
    def training_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']
        mask = (inputs != self.hparams.input_padding_idx).byte()
        # We receive one batch of data and perform a forward pass:
        logits, predictions = self.forward(inputs, mask)

        loss = self.crf(logits, labels, mask, reduction='mean')
        loss = -loss # The loss returned by the CRF module is not negative

        predictions = [torch.LongTensor(prediction) for prediction in predictions]
        predictions = pad_sequence(predictions, batch_first=True, padding_value=self.hparams.label_padding_idx)
        labels = labels.view(-1)

        F1 = self.F1(predictions.view(-1), labels)
        # Log it:
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_f1', F1, prog_bar=True)
        # Very important for PL to return the loss that will be used to update the weights:
        return loss


    # This runs the model in eval mode, ie. sets dropout to 0 and deactivates grad. Needed when we are in inference mode.
    def validation_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']
        mask = (inputs != self.hparams.input_padding_idx).byte()

        logits, predictions = self.forward(inputs, mask)

        sample_loss = self.crf(logits, labels, mask, reduction='mean')
        sample_loss = -sample_loss # The loss returned by the CRF module is not negative

        predictions = [torch.LongTensor(prediction) for prediction in predictions]
        predictions = pad_sequence(predictions, batch_first=True, padding_value=self.hparams.label_padding_idx)
        labels = labels.view(-1)

        sample_F1 = self.F1(predictions.view(-1), labels)

        self.log('valid_loss', sample_loss, prog_bar=True)
        self.log('valid_f1', sample_F1, prog_bar=True)

    # This runs the model in eval mode, ie. sets dropout to 0 and deactivates grad. Needed when we are in inference mode.
    def test_step(self, batch, batch_nb):
        inputs = batch['inputs']
        # Compute lengths of sequences, which we will need
        mask = (inputs != self.hparams.input_padding_idx).byte()
        labels = batch['outputs']

        logits, predictions = self.forward(inputs, mask)
        decoded_labels = ABSA_A_Dataset.decode_output(predictions, self.hparams.label_vocabulary)
        scores, precision, recall, f1 = evaluate_aspect_terms(decoded_labels, batch['text'], batch['indexes'], batch['true_aspect_terms'])
        sample_loss = self.crf(logits, labels, mask, reduction='mean')
        sample_loss = -sample_loss # The loss returned by the CRF module is not negative

        predictions = [torch.LongTensor(prediction) for prediction in predictions]
        predictions = pad_sequence(predictions, batch_first=True, padding_value=self.hparams.label_padding_idx)
        labels = labels.view(-1)

        sample_F1 = self.F1(predictions.view(-1), labels)
        self.log('test_loss', sample_loss, prog_bar=True)
        self.log('test_f1_on_NE_labels', sample_F1, prog_bar=True)
        self.log('test_tp', scores['tp'], prog_bar=True)
        self.log('test_fp', scores['fp'], prog_bar=True)
        self.log('test_fn', scores['fn'], prog_bar=True)
        self.log('test_precision', precision, prog_bar=True)
        self.log('test_recall', recall, prog_bar=True)
        self.log('test_f1', f1, prog_bar=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters())

In [ ]:
pretrained_embeddings = torch.randn(len(vocabulary), vectors.dim)
initialised = 0
for i, w in enumerate(vocabulary.itos):
    if w in vectors.stoi:
        initialised += 1
        vec = vectors.get_vecs_by_tokens(w)
        pretrained_embeddings[i] = vec
    
pretrained_embeddings[vocabulary["<pad>"]] = torch.zeros(vectors.dim)
print("initialised embeddings {}".format(initialised))
print("random initialised embeddings {} ".format(len(vocabulary) - initialised))
print(pretrained_embeddings)
print(pretrained_embeddings.shape)
EMBEDDING_DIM = vectors.dim
vectors = None # To free up RAM

initialised embeddings 2196016
random initialised embeddings 2 
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.0826e-01, -7.3776e-01, -3.2175e-01,  ...,  1.5567e-01,
          6.4575e-01, -8.0658e-01],
        [ 2.7204e-01, -6.2030e-02, -1.8840e-01,  ...,  1.3015e-01,
         -1.8317e-01,  1.3230e-01],
        ...,
        [ 9.0535e-01,  5.4222e-01,  5.3270e-02,  ...,  1.3901e-01,
         -5.7334e-01,  5.5285e-02],
        [-1.6985e-04, -1.2435e-03,  2.7223e-03,  ...,  7.1386e-04,
         -1.3968e-03, -1.5977e-03],
        [ 4.3977e-04, -1.0896e-03,  8.8869e-04,  ...,  1.9686e-04,
         -9.4193e-04,  5.0061e-04]])
torch.Size([2196018, 300])


## Model Training

In [ ]:
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  # the value that will be evaluated to activate the early stopping of the model.
    patience=10,  # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='min', # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='min',  # wheter we want to maximize (max) or minimize the "monitor" value.
    dirpath='experiments/Aspect_Term_Classifier',  # output directory path
    filename='{epoch}-{valid_loss:.4f}'  # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)

hparams = {'vocab_size': len(vocabulary),
            'hidden_dim': 128,
            'embedding_dim': EMBEDDING_DIM,
            'num_classes': len(label_vocabulary), # number of different NE labels in our case
            'bidirectional': True,
            'num_layers': 2,
            'dropout': 0.0,
            'input_padding_idx': vocabulary["<pad>"],
            'label_padding_idx': label_vocabulary["<pad>"],
            'label_vocabulary': label_vocabulary}

data_module = DataModuleABSA_A(train_dataframe['targets'], train_dataframe['text'], valid_dataframe['targets'], valid_dataframe['text'], vocabulary, label_vocabulary)
train_dataframe = None
valid_dataframe = None

trainer = pl.Trainer(gpus=1 if torch.cuda.is_available() else 0,
                     val_check_interval=1.0,
                     max_epochs=25,
                     deterministic=True,
                     callbacks=[early_stopping, check_point_callback] # the callback we want our trainer to use.
)

model = AspectTermModule(hparams, pretrained_embeddings)
pretrained_embeddings = None
trainer.fit(model, datamodule=data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory experiments/Aspect_Term_Classifier exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


{'bidirectional': True,
 'dropout': 0.0,
 'embedding_dim': 300,
 'hidden_dim': 128,
 'input_padding_idx': 0,
 'label_padding_idx': 0,
 'label_vocabulary': <torchtext.vocab.Vocab object at 0x7f54ba1cc2d0>,
 'num_classes': 3,
 'num_layers': 2,
 'vocab_size': 2196018}
initializing embeddings from pretrained



  | Name  | Type                          | Params
--------------------------------------------------------
0 | F1    | F1                            | 0     
1 | model | AspectTermIdentificationModel | 659 M 
2 | crf   | CRF                           | 15    
--------------------------------------------------------
836 K     Trainable params
658 M     Non-trainable params
659 M     Total params
2,638.567 Total estimated model params size (MB)


Global seed set to 1234


Metric valid_loss improved. New best score: 2.987
Epoch 0, global step 39: valid_loss reached 2.98669 (best 2.98669), saving model to "/content/experiments/Aspect_Term_Classifier/epoch=0-valid_loss=2.9867.ckpt" as top 1


...

Epoch 13, global step 559: valid_loss was not in top 1


Epoch 14, global step 599: valid_loss was not in top 1


Epoch 15, global step 639: valid_loss was not in top 1


Epoch 16, global step 679: valid_loss was not in top 1


Monitored metric valid_loss did not improve in the last 10 records. Best score: 1.443. Signaling Trainer to stop.
Epoch 17, global step 719: valid_loss was not in top 1


## Model Evaluation

In [ ]:
model = AspectTermModule.load_from_checkpoint("/content/experiments/Aspect_Term_Classifier/epoch=7-valid_loss=1.4433.ckpt")  # REPLACE HERE THE PATH TO A MODEL CHECKPOINT IN THE "experiments/Aspect_Term_Classifier" folder

test_set_results = trainer.test(model, test_dataloaders=data_module.test_dataloader())
print("test set results: {}".format(test_set_results))

{'bidirectional': True,
 'dropout': 0.0,
 'embedding_dim': 300,
 'hidden_dim': 128,
 'input_padding_idx': 0,
 'label_padding_idx': 0,
 'label_vocabulary': <torchtext.vocab.Vocab object at 0x7f546e6b8790>,
 'num_classes': 3,
 'num_layers': 2,
 'vocab_size': 2196018}



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 72.68722466960352,
 'test_f1_on_NE_labels': 0.8968632221221924,
 'test_fn': 257.0,
 'test_fp': 363.0,
 'test_loss': 1.4432944059371948,
 'test_precision': 69.44444444444444,
 'test_recall': 76.24768946395564,
 'test_tp': 825.0}
--------------------------------------------------------------------------------
test set results: [{'test_loss': 1.4432944059371948, 'test_f1_on_NE_labels': 0.8968632221221924, 'test_tp': 825.0, 'test_fp': 363.0, 'test_fn': 257.0, 'test_precision': 69.44444444444444, 'test_recall': 76.24768946395564, 'test_f1': 72.68722466960352}]
